<a href="https://colab.research.google.com/github/taitip-supha/insurance-propensity-model/blob/main/insurance_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup Notebook

In [1]:
%pip install bayesian-optimization
%pip install lightgbm==3.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests

import pyspark
from pyspark.sql import DataFrame, SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# package for develop model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import lightgbm as lgb
import shap

from google.colab.data_table import DataTable
DataTable.max_columns = 50
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
spark= SparkSession.builder.appName("insurance model").getOrCreate()

#Import data

In [4]:
# load data from googleDrive
spk_ins = spark.read.csv("/gdrive/MyDrive/insProject/dsc_test_case.csv", header=True, inferSchema=True)\
                      .withColumn('campaign_month',  F.month(F.to_date(F.col('campaign_month') , 'MMM') ))

col_cate = ['campaign_month', 'marital_sta', 'main_occupation', 'customer_segment', 'gender', 'have_acc_planet', 'have_cc', 'payroll']
col_num = ['num_children', 'age', 'income', 'maxosdc_last_30d', 'dcspend_last_30d', 'easypymt_last_30d', 'savacc_bal', 'currentacc_bal', 'avg_savaccbal_30d', 'avg_currentaccbal_30d', 'mob', 'inflow30d', 'outflow30d', 'inflow1_15', 'outflow1_15', 'net_flow_30d', 'net_flow_15d',]
print(f"total record in dsc_test_case : {spk_ins.count():0,d} row")
display(spk_ins.limit(10).toPandas())

total record in dsc_test_case : 215,993 row


,campaign_month,marital_sta,main_occupation,customer_segment,gender,have_acc_planet,have_cc,payroll,num_children,age,income,maxosdc_last_30d,dcspend_last_30d,easypymt_last_30d,savacc_bal,currentacc_bal,avg_savaccbal_30d,avg_currentaccbal_30d,mob,inflow30d,outflow30d,inflow1_15,outflow1_15,net_flow_30d,net_flow_15d,label
0,3,โสด,Salary man,Lower Mass,Female,N,N,Y,0.0,41.0,11017.00,15242.93,0.0,210.0,6442.93,0.0,2034.46,0.0,77.0,0.00,0.00,0.00,0.00,0.00,0.00,0
1,7,สมรส,Salary man,Mass,Female,N,N,N,1.0,46.0,15140.15,0.00,0.0,0.0,0.00,0.0,0.00,0.0,14.0,0.00,0.00,0.00,0.00,0.00,0.00,0
2,1,โสด,Self-employed,Lower Mass,Male,N,N,N,0.0,36.0,452.87,1205.00,0.0,1486.0,1099.71,0.0,217.77,0.0,37.0,13558.67,14454.97,13558.67,14454.97,-896.30,-896.30,0
3,11,โสด,Salary man,Mass,Female,N,N,N,0.0,38.0,12036.17,0.00,0.0,0.0,45554.61,0.0,45554.61,0.0,165.0,0.00,0.00,0.00,0.00,0.00,0.00,0
4,10,สมรสจด ทะเบียน,Salary man,Mass,Female,N,N,N,0.0,41.0,25000.00,1896.51,0.0,5900.0,1224.51,0.0,1238.59,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0
5,7,โสด,Freelance,Mass,Male,N,N,N,0.0,37.0,15140.15,0.00,0.0,0.0,1.49,0.0,1.49,0.0,10.0,0.00,0.00,0.00,0.00,0.00,0.00,0
6,3,สมรส,Self-employed,Mass,Male,N,N,N,0.0,39.0,16389.58,0.00,0.0,727.0,768786.07,0.0,270783.58,0.0,67.0,0.00,0.00,0.00,0.00,0.00,0.00,0
7,11,โสด,Housewife,Mass,Female,N,N,N,2.0,43.0,15999.69,1082.11,0.0,1070.0,5.31,0.0,115.42,0.0,62.0,32428.00,32431.85,32428.00,32431.85,-3.85,-3.85,0
8,3,โสด,Self-employed,Lower Mass,Male,N,N,N,0.0,27.0,4364.98,0.00,0.0,653.0,4.51,0.0,4.51,0.0,6.0,0.00,0.00,0.00,0.00,0.00,0.00,0
9,7,สมรส,Salary man,Upper Mass,Female,N,N,N,0.0,42.0,67677.40,0.00,0.0,5429.0,3002.85,0.0,223.47,0.0,55.0,6998.00,8901.85,6998.00,8901.85,-1903.85,-1903.85,0


In [5]:
display(spk_ins.summary().toPandas().set_index('summary').T)

summary,count,mean,stddev,min,25%,50%,75%,max
campaign_month,215993,6.407522466005843,3.79413303244298,1,3,6,10,12
marital_sta,211358,None,None,ม่าย,None,None,None,โสด
main_occupation,214493,None,None,Entertainer,None,None,None,Unemployed
customer_segment,214493,None,None,Lower Mass,None,None,None,Upper Mass
gender,214493,None,None,Female,None,None,None,Male
have_acc_planet,213592,None,None,N,None,None,None,Y
have_cc,214493,None,None,N,None,None,None,Y
payroll,214493,None,None,N,None,None,None,Y
num_children,214493,0.04038826441888547,0.25005003383234203,0.0,0.0,0.0,0.0,21.0
age,214493,37.78451511238129,9.589736644865821,23.0,30.0,36.0,45.0,61.0


#Explore data

In [6]:
for _c in col_cate:
    _tmp = spk_ins.groupBy(_c).agg( F.count('*').alias('count')
                                                  , F.mean(F.when(F.col('label').isin(1,2), 1).otherwise(0)).alias('accept_ins_offer')
                                                  , F.mean(F.when(F.col('label').isin(1), 1).otherwise(0)).alias('accept_pa_offer')
                                                  , F.mean(F.when(F.col('label').isin(2), 1).otherwise(0)).alias('accept_life_offer') )
    print(f"\nexplore column {_c}")
    display(_tmp.orderBy(_c).toPandas())
# None 1500 : scb_payroll, have_cc, gender, customer_segment, main_occupation
# columns scb_payroll, have_cc, have_acc_planet : None = N
# columns main_occupation : None = ??
# columns customer_segment : None = ??
# marital_sta : None = ?? ['สมรส', 'สมรสจด ทะเบียน', 'สมรสไม่ จดทะเบียน', 'ม่าย', 'แยกกันอยู่','หย่าร้าง']
#                                 , ['โสด', 'แยกกันอยู่', 'ม่าย', 'แยกกันอยู่','หย่าร้าง']
# gender : None = ??


explore column campaign_month


,campaign_month,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,1,22282,0.012970,0.007450,0.005520
1,2,22264,0.007995,0.002830,0.005165
2,3,21937,0.008479,0.003191,0.005288
3,4,21933,0.010942,0.006292,0.004651
4,5,14672,0.012268,0.009269,0.002999
5,6,10179,0.017487,0.013066,0.004421
6,7,11242,0.019036,0.012186,0.006849
7,8,11107,0.019537,0.014855,0.004682
8,9,11210,0.018912,0.011686,0.007226
9,10,23625,0.010667,0.007873,0.002794



explore column marital_sta


,marital_sta,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,None,4635,0.007767,0.005394,0.002373
1,ม่าย,946,0.011628,0.009514,0.002114
2,สมรส,45892,0.009871,0.006864,0.003007
3,สมรสจด ทะเบียน,6941,0.009509,0.005187,0.004322
4,สมรสไม่ จดทะเบียน,854,0.011710,0.005855,0.005855
5,หย่าร้าง,2627,0.010278,0.007994,0.002284
6,อื่นๆ,1452,0.010331,0.007576,0.002755
7,แยกกันอยู่,111,0.018018,0.009009,0.009009
8,โสด,152535,0.013125,0.007939,0.005186



explore column main_occupation


...

,have_acc_planet,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,None,2401,0.002499,0.000416,0.002082
1,N,210167,0.012319,0.007737,0.004582
2,Y,3425,0.007883,0.002044,0.005839



explore column have_cc


,have_cc,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,None,1500,0.000667,0.000667,0.000000
1,N,193066,0.012503,0.008106,0.004397
2,Y,21427,0.009661,0.003174,0.006487



explore column payroll


,payroll,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,None,1500,0.000667,0.000667,0.000000
1,N,170729,0.011773,0.007374,0.004399
2,Y,43764,0.013961,0.008546,0.005415


In [7]:
for i,_c in enumerate(col_num):
      percentiles = spk_ins.approxQuantile(_c, [0.00, 0.25, 0.5, 0.75, 1.00], 0.01)
      _min, _q1, _q2, _q3, _max = percentiles
      print(f"\n{i+1}/{len(col_num)} - '{_c}' percentile :_min:{_min:,.0f}, P25:{_q1:,.0f}, P50:{_q2:,.0f}, P75:{_q3:,.0f}, _max:{_max:,.0f}")
      spk_ins_grouped = spk_ins.withColumn(f'{_c}_group', F.when(F.col(_c) <= _q1, f'01-Below {_q1:.0f}')
                                                                                   .when(F.col(_c) <= _q2, f'02-Between {_q1:.0f} and {_q2:.0f}')
                                                                                   .when(F.col(_c) <= _q3, f'03-Between{_q2:.0f} and {_q3:.0f}')
                                                                                   .otherwise(f'04-Above {_q3:.0f}'))
      _tmp = spk_ins_grouped.groupBy(f'{_c}_group').agg(
                                            F.count('*').alias('count')
                                          , F.mean(F.when(F.col('label').isin(1,2), 1).otherwise(0)).alias('accept_ins_offer')
                                          , F.mean(F.when(F.col('label').isin(1), 1).otherwise(0)).alias('accept_pa_offer')
                                          , F.mean(F.when(F.col('label').isin(2), 1).otherwise(0)).alias('accept_life_offer') )
      display(_tmp.orderBy(f'{_c}_group').toPandas())


1/17 - 'num_children' percentile :_min:0, P25:0, P50:0, P75:0, _max:21


,num_children_group,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,01-Below 0,207701,0.012277,0.007602,0.004675
1,04-Above 0,8292,0.008683,0.006633,0.002050



2/17 - 'age' percentile :_min:23, P25:30, P50:36, P75:45, _max:61


,age_group,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,01-Below 30,61527,0.016789,0.010223,0.006566
1,02-Between 30 and 36,48558,0.011697,0.006487,0.005210
2,03-Between36 and 45,54521,0.009556,0.005869,0.003687
3,04-Above 45,51387,0.009730,0.007200,0.002530



3/17 - 'income' percentile :_min:0, P25:4,365, P50:12,203, P75:21,000, _max:199,626


...

,outflow1_15_group,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,01-Below 0,110963,0.007219,0.002866,0.004353
1,03-Between0 and 17161,47376,0.015514,0.011082,0.004433
2,04-Above 17161,57654,0.018837,0.013720,0.005117



16/17 - 'net_flow_30d' percentile :_min:-1,813,780, P25:-1, P50:0, P75:0, _max:2,295,697


,net_flow_30d_group,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,01-Below -1,52010,0.017823,0.012748,0.005076
1,02-Between -1 and 0,115176,0.007580,0.003239,0.004341
2,04-Above 0,48807,0.016842,0.012252,0.004590



17/17 - 'net_flow_15d' percentile :_min:-1,813,780, P25:-1, P50:0, P75:0, _max:2,295,697


,net_flow_15d_group,count,accept_ins_offer,accept_pa_offer,accept_life_offer
0,01-Below -1,52010,0.017823,0.012748,0.005076
1,02-Between -1 and 0,115176,0.007580,0.003239,0.004341
2,04-Above 0,48807,0.016842,0.012252,0.004590


#Prepare data

## Impute data and Feture Engineer

In [8]:
#impute Null
spk_impute = spk_ins.withColumn('obs_id', F.row_number().over(Window().orderBy(F.lit('A'))))\
                                .withColumn('target', F.when(F.col('label').isin(1,2), 1).otherwise(0) )\
                                .withColumn('ntu_flag', F.when(F.col('customer_segment').isNull(), 1).otherwise(0))\
                                .withColumn('payroll', F.when(F.col('payroll')=='Y', 1).otherwise(0))\
                                .withColumn('have_cc', F.when(F.col('have_cc')=='Y', 1).otherwise(0) )\
                                .withColumn('have_acc_planet', F.when(F.col('have_acc_planet')=='Y', 1).otherwise(0) )\
                                .withColumn('occ_sa',F.when(F.col('main_occupation')=='Salary man', 1).otherwise(0) )\
                                .withColumn('occ_se', F.when(F.col('main_occupation')=='Self-employed', 1).otherwise(0) )\
                                .withColumn('occ_gov', F.when(F.col('main_occupation')=='Government', 1).otherwise(0) )\
                                .withColumn('occ_prof', F.when(F.col('main_occupation')=='Professional', 1).otherwise(0) )\
                                .withColumn('occ_ent', F.when(F.col('main_occupation')=='Entertainer', 1).otherwise(0) )\
                                .withColumn('occ_frl', F.when(F.col('main_occupation')=='Freelance', 1).otherwise(0) )\
                                .withColumn('occ_hwf', F.when(F.col('main_occupation')=='Housewife', 1).otherwise(0) )\
                                .withColumn('occ_stu', F.when(F.col('main_occupation')=='Student', 1).otherwise(0) )\
                                .withColumn('occ_une', F.when(F.col('main_occupation')=='Unemployed', 1).otherwise(0) )\
                                .withColumn('seg_lmass', F.when(F.col('customer_segment')=='Lower Mass', 1).otherwise(0) )\
                                .withColumn('seg_mass', F.when(F.col('customer_segment')=='Mass', 1).otherwise(0) )\
                                .withColumn('seg_umass', F.when(F.col('customer_segment')=='Upper Mass', 1).otherwise(0) )\
                                .withColumn('gender_female', F.when( F.col('gender')=='Female', 1).otherwise(0))\
                                .withColumn('sta_single', F.when( F.col('marital_sta')=='โสด', 1).otherwise(0))\
                                .withColumn('avgsav2incm', F.col('avg_savaccbal_30d')/F.col('income') )\
                                .withColumn('avgsav2inflow', F.col('avg_savaccbal_30d')/F.col('inflow30d') )\
                                .withColumn("debitspend2inflow", F.col("dcspend_last_30d") / F.col("inflow30d")) \
                                .withColumn("savings_ratio", F.col("savacc_bal") / (F.col("savacc_bal") + F.col("currentacc_bal"))) \
                                .withColumn("sav_stability", F.col("avg_savaccbal_30d")/F.col("savacc_bal") ) \
                                .withColumn("netflow2inflow", F.col("net_flow_30d")/F.col("inflow30d") )\
                                .withColumn("bank_sav",  F.when(F.col('avg_savaccbal_30d')>0, 1).otherwise(0) )\
                                .withColumn("bank_cur",  F.when(F.col('avg_currentaccbal_30d')>0, 1).otherwise(0) )\
                                .withColumn("bank_prod", F.col("payroll")+F.col('have_cc')+F.col('have_acc_planet')+F.col('bank_sav')+F.col('bank_cur')) \
                                .fillna(-9999999, subset=col_num)\
                                .cache()
display(spk_impute.summary().toPandas().T)




,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
campaign_month,215993,6.407522466005843,3.794133032442977,1,3,6,10,12
marital_sta,211358,None,None,ม่าย,None,None,None,โสด
main_occupation,214493,None,None,Entertainer,None,None,None,Unemployed
customer_segment,214493,None,None,Lower Mass,None,None,None,Upper Mass
gender,214493,None,None,Female,None,None,None,Male
have_acc_planet,215993,0.015856995365590553,0.12492246921256231,0,0,0,0,1
have_cc,215993,0.09920228896306825,0.2989341207559202,0,0,0,0,1
payroll,215993,0.20261767742473136,0.40195087041551,0,0,0,0,1
num_children,215993,-69446.64797933266,830449.9788809681,-9999999.0,0.0,0.0,0.0,21.0


##Split data train/test/out-of-time

In [9]:
def spkSelect(spk_from, lst_tm, lst_feat):
    spk_return = spk_from.filter(F.col('campaign_month').isin(lst_tm)).select(['obs_id','target']+lst_feat)
    print(f"select data campaign_month in {lst_tm}): {spk_return.count():0,d} row, {len(spk_return.columns)} columns (with-target)")
    return spk_return

col_feat = col_num+['payroll','have_cc','have_acc_planet','ntu_flag'
                              ,'occ_sa','occ_se','occ_gov','occ_prof','occ_ent','occ_frl','occ_hwf','occ_stu','occ_une'
                              ,'seg_lmass','seg_mass','seg_umass','gender_female','sta_single'
                              ,'avgsav2incm','avgsav2inflow','debitspend2inflow','savings_ratio','sav_stability'
                              ,'netflow2inflow','bank_sav','bank_cur','bank_prod',  ]
spk_dev = spkSelect(spk_from=spk_impute, lst_tm=[_m for _m in range(1,12)], lst_feat=col_feat)
spk_oot = spkSelect(spk_from=spk_impute, lst_tm=[12], lst_feat=col_feat)

# Split develop data into train and test sets
spk_train, spk_test = spk_dev.randomSplit([0.8, 0.2], seed=42)
spk_train = spk_train.cache()
spk_test = spk_test.cache()
print(f"Train data : {spk_train.count():0,d} row, {len(spk_train.columns)} columns ({spk_train.count()/spk_dev.count():0.0%})")
print(f"Test data : {spk_test.count():0,d} row, {len(spk_test.columns)} columns ({spk_test.count()/spk_dev.count():0.0%})")


select data campaign_month in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]): 192,452 row, 46 columns (with-target)
select data campaign_month in [12]): 23,541 row, 46 columns (with-target)
Train data : 153,803 row, 46 columns (80%)
Test data : 38,649 row, 46 columns (20%)


## Under Sampling

In [10]:
#get size of undersampling
positive_target = 0.05
positive_record =  spk_train.filter(F.col('target') == 1).count()
positive_current = positive_record/spk_train.count()
nagative_record =  spk_train.filter(F.col('target') == 0).count()
nagative_new = int((positive_record -positive_target*positive_record)/positive_target +0.5)
print(f"%Positive class (1) -> current {positive_current:.5f} -> target : {positive_target:.5f} effect to negative class reduct record to {nagative_new:,d} row ({nagative_new/nagative_record:0.0%})")

spk_negative = spk_train.filter(F.col('target') == 0).sample(withReplacement=False, fraction=nagative_new/nagative_record, seed=123)
spk_train_bal = spk_train.filter(F.col('target') == 1).unionByName(spk_negative).orderBy(F.rand())
print(f'New data balance class pos-{positive_target} have {spk_train_bal.count():,d} row ({spk_train_bal.count()/spk_train.count():0.0%})')

%Positive class (1) -> current 0.01225 -> target : 0.05000 effect to negative class reduct record to 35,796 row (24%)
New data balance class pos-0.05 have 37,539 row (24%)


## Data in Pandas Format

In [11]:
pd_dev = spk_dev.toPandas().reset_index(drop=True)
pd_train = spk_train_bal.toPandas().reset_index(drop=True)
pd_test = spk_test.toPandas().reset_index(drop=True)
pd_oot = spk_oot.toPandas().reset_index(drop=True)
print(f"pd_dev data for Develop model : {pd_dev.shape}")
print(f"pd_train data for training model : {pd_train.shape}")
print(f"pd_test data for testing model : {pd_test.shape}")
print(f"pd_oot data for test out-of-time model : {pd_oot.shape}")

pd_dev data for Develop model : (192452, 46)
pd_train data for training model : (37539, 46)
pd_test data for testing model : (38649, 46)
pd_oot data for test out-of-time model : (23541, 46)


#Model Development

## Shallow tree

In [12]:
df_feature = spk_train_bal.summary().toPandas().T.reset_index()
df_feature = df_feature.rename(columns=df_feature.iloc[0]).drop(df_feature.index[0]).rename(columns={'summary':'feature'})
print(f"Feature start")

# summary	count	mean	stddev	min	25%	50%	75%	max

Feature start


,feature,count,mean,stddev,min,25%,50%,75%,max
1,obs_id,37539,107588.72974772903,62377.74124081243,1,53589,107246,161656,215993
2,target,37539,0.05018780468312955,0.2183351983314606,0,0,0,0,1
3,num_children,37539,-63134.293428168035,792069.4730762646,-9999999.0,0.0,0.0,0.0,5.0
4,age,37539,-63096.861131090336,792072.4568699166,-9999999.0,29.0,36.0,45.0,61.0
5,income,37539,-92202.79700657756,1041360.0828098088,-9999999.0,4364.98,12324.44,21345.0,198000.0
6,maxosdc_last_30d,37539,-97975.68425743919,1042180.3773554043,-9999999.0,0.0,0.0,4056.29,2084410.06
7,dcspend_last_30d,37539,-108994.38270065801,1039453.6403301145,-9999999.0,0.0,0.0,0.0,1541316.58
8,easypymt_last_30d,37539,-108074.22467300674,1039518.4874098451,-9999999.0,77.0,374.0,925.0,999982.0
9,savacc_bal,37539,-89281.38697621174,1045652.6419708276,-9999999.0,5.63,768.6,7476.5,4414296.66
10,currentacc_bal,37539,-109219.73259809798,1039373.3965960342,-9999999.0,0.0,0.0,0.0,1.0


In [13]:
def pdPredict(_df, _model, _nbin=10, _iteration=None):
  _feat = _model.feature_name()
  pred_prob = _model.predict(_df[_feat], num_iteration=_iteration)
  pred_bin = pd.qcut(pred_prob, _nbin, labels=[c+1 for c in range(_nbin)])
  roc_auc = roc_auc_score(_df['target'], pred_prob)
  print(f'predict data {_df.shape[0]} row by using {len(_feat)} feature [roc/auc={roc_auc:0.3%}, gini={2*roc_auc-1:0.3%}]')
  return pred_prob, pred_bin

In [14]:
#Training Shallow tree model
lgb_train = lgb.Dataset(pd_train[col_feat], label=pd_train['target'])
lgb_eval = lgb.Dataset(pd_test[col_feat], label=pd_test['target'], reference=lgb_train)

params = { 'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'max_depth':2, 'random_state': 123 , 'verbosity': -1 } #
print(f"Training  Shallow tree model with {len(col_feat)} feature : {col_feat} \nwith parameter :{params}")
shallow_tree = lgb.train(params,  lgb_train,  valid_sets=[lgb_eval])

print("\n\nPredict and Model Performance")
pd_train['pred_prob'], pd_train['pred_bin'] = pdPredict(_df=pd_train, _model=shallow_tree)
pd_test['pred_prob'], pd_test['pred_bin'] = pdPredict(_df=pd_test, _model=shallow_tree)
pd_oot['pred_prob'], pd_oot['pred_bin'] = pdPredict(_df=pd_oot, _model=shallow_tree)

_feature = pd.DataFrame({'feature':shallow_tree.feature_name() , 'importance':shallow_tree.feature_importance()}).sort_values('importance', ascending=False)
df_feature = df_feature.merge(_feature, on='feature', how='left')
print(f"\n\nfeature importance from initial model (importance =0 : {df_feature[df_feature.importance==0].shape[0]}) ({np.mean(df_feature.importance==0):0.2%})")
display(df_feature)

Training  Shallow tree model with 44 feature : ['num_children', 'age', 'income', 'maxosdc_last_30d', 'dcspend_last_30d', 'easypymt_last_30d', 'savacc_bal', 'currentacc_bal', 'avg_savaccbal_30d', 'avg_currentaccbal_30d', 'mob', 'inflow30d', 'outflow30d', 'inflow1_15', 'outflow1_15', 'net_flow_30d', 'net_flow_15d', 'payroll', 'have_cc', 'have_acc_planet', 'ntu_flag', 'occ_sa', 'occ_se', 'occ_gov', 'occ_prof', 'occ_ent', 'occ_frl', 'occ_hwf', 'occ_stu', 'occ_une', 'seg_lmass', 'seg_mass', 'seg_umass', 'gender_female', 'sta_single', 'avgsav2incm', 'avgsav2inflow', 'debitspend2inflow', 'savings_ratio', 'sav_stability', 'netflow2inflow', 'bank_sav', 'bank_cur', 'bank_prod'] 
with parameter :{'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt', 'max_depth': 2, 'random_state': 123, 'verbosity': -1}
[1]	valid_0's auc: 0.624225
[2]	valid_0's auc: 0.64036
[3]	valid_0's auc: 0.64843
[4]	valid_0's auc: 0.648069
[5]	valid_0's auc: 0.651191
[6]	valid_0's auc: 0.651929
[7]	valid_0's auc: 0

,feature,count,mean,stddev,min,25%,50%,75%,max,importance
0,obs_id,37539,107588.72974772903,62377.74124081243,1,53589,107246,161656,215993,NaN
1,target,37539,0.05018780468312955,0.2183351983314606,0,0,0,0,1,NaN
2,num_children,37539,-63134.293428168035,792069.4730762646,-9999999.0,0.0,0.0,0.0,5.0,1.0
3,age,37539,-63096.861131090336,792072.4568699166,-9999999.0,29.0,36.0,45.0,61.0,49.0
4,income,37539,-92202.79700657756,1041360.0828098088,-9999999.0,4364.98,12324.44,21345.0,198000.0,10.0
5,maxosdc_last_30d,37539,-97975.68425743919,1042180.3773554043,-9999999.0,0.0,0.0,4056.29,2084410.06,18.0
6,dcspend_last_30d,37539,-108994.38270065801,1039453.6403301145,-9999999.0,0.0,0.0,0.0,1541316.58,11.0
7,easypymt_last_30d,37539,-108074.22467300674,1039518.4874098451,-9999999.0,77.0,374.0,925.0,999982.0,18.0
8,savacc_bal,37539,-89281.38697621174,1045652.6419708276,-9999999.0,5.63,768.6,7476.5,4414296.66,13.0
9,currentacc_bal,37539,-109219.73259809798,1039373.3965960342,-9999999.0,0.0,0.0,0.0,1.0,0.0


##Feature Selection

In [15]:
col_cal_corr = col_feat; lst_corr = list()
print(f"calculate spearman-correlation : {len(col_cal_corr)*(len(col_cal_corr)+1)/2} Case, (reduce process corr(a,b) = corr(b,a) )")
for i,colX in enumerate(tqdm(col_cal_corr)):
    for coly in col_cal_corr[i:]:
        if colX==coly : corr_score=1
        else :
            try :
                corr_score = pd_dev[colX].corr(pd_dev[coly], method='spearman')
                lst_corr.append([colX, coly, round(corr_score,6)])
            except :
                print('error',colX,coly)
feat_corr = pd.DataFrame(lst_corr,columns=['col_x','col_y','corr_score'])
display(feat_corr.head(10))

calculate spearman-correlation : 990.0 Case, (reduce process corr(a,b) = corr(b,a) )


 45%|████▌     | 20/44 [00:13<00:11,  2.05it/s]/usr/local/lib/python3.11/dist-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]
100%|██████████| 44/44 [00:18<00:00,  2.37it/s]


,col_x,col_y,corr_score
0,num_children,age,0.232189
1,num_children,income,0.038814
2,num_children,maxosdc_last_30d,0.087813
3,num_children,dcspend_last_30d,0.137943
4,num_children,easypymt_last_30d,0.051507
5,num_children,savacc_bal,0.056068
6,num_children,currentacc_bal,0.283555
7,num_children,avg_savaccbal_30d,0.061779
8,num_children,avg_currentaccbal_30d,0.287508
9,num_children,mob,0.045881


In [16]:
lst_select = list(); lst_process = list(); corr_cutoff = 0.75
for _feat in col_cal_corr:
    _left =  list(feat_corr[(feat_corr.col_x == _feat) & (feat_corr.corr_score.abs()>=corr_cutoff)]['col_y'])
    _right = list(feat_corr[(feat_corr.col_y == _feat) & (feat_corr.corr_score.abs()>=corr_cutoff)]['col_x'])
    _corrs = list(set(_left+_right+[_feat]))
    if len(_corrs)>0 :
        _impor = list(df_feature[df_feature.feature.isin(_corrs)].sort_values(by=['importance','stddev'] , ascending=False)['feature'])[0]
    if _impor not in lst_process:
        lst_select.append(_impor)
    lst_process.extend(_corrs)
    _left = _right = _corrs = None
lst_select = list(set(lst_select))
lst_process = list(set(lst_process))
df_feature['selected'] = df_feature.feature.apply(lambda x: 1 if x in lst_select else 0)
print(f"Selected {sum(df_feature.selected)} feature / ({len(df_feature)}) ")
display(df_feature)

Selected 32 feature / (46) 


,feature,count,mean,stddev,min,25%,50%,75%,max,importance,selected
0,obs_id,37539,107588.72974772903,62377.74124081243,1,53589,107246,161656,215993,NaN,0
1,target,37539,0.05018780468312955,0.2183351983314606,0,0,0,0,1,NaN,0
2,num_children,37539,-63134.293428168035,792069.4730762646,-9999999.0,0.0,0.0,0.0,5.0,1.0,1
3,age,37539,-63096.861131090336,792072.4568699166,-9999999.0,29.0,36.0,45.0,61.0,49.0,1
4,income,37539,-92202.79700657756,1041360.0828098088,-9999999.0,4364.98,12324.44,21345.0,198000.0,10.0,1
5,maxosdc_last_30d,37539,-97975.68425743919,1042180.3773554043,-9999999.0,0.0,0.0,4056.29,2084410.06,18.0,1
6,dcspend_last_30d,37539,-108994.38270065801,1039453.6403301145,-9999999.0,0.0,0.0,0.0,1541316.58,11.0,1
7,easypymt_last_30d,37539,-108074.22467300674,1039518.4874098451,-9999999.0,77.0,374.0,925.0,999982.0,18.0,1
8,savacc_bal,37539,-89281.38697621174,1045652.6419708276,-9999999.0,5.63,768.6,7476.5,4414296.66,13.0,0
9,currentacc_bal,37539,-109219.73259809798,1039373.3965960342,-9999999.0,0.0,0.0,0.0,1.0,0.0,0


##Hyperparameter tuning

In [17]:
from bayes_opt import BayesianOptimization
import warnings
def SetParameter( max_depth, num_leaves, min_child_samples, bagging_fraction, bagging_freq, feature_fraction, reg_alpha, reg_lambda, learning_rate, num_iterations, early_stopping_rounds):
    params = {'boosting_type': 'gbdt', 'objective': 'binary', 'metric':['auc'], 'verbose':-1}
    params['max_depth'] = int(round(max_depth)) #int
    params['num_leaves'] = int(round(num_leaves)) #int
    params['min_child_samples'] = int(round(min_child_samples,4)) #int
    params['bagging_fraction'] = round(max(bagging_fraction,0),4) #double
    params['bagging_freq'] = int(round(bagging_freq)) #int
    params['feature_fraction'] = round(max(feature_fraction,0),4) #double
    params['reg_alpha'] = round(max(reg_alpha,0),4) #double
    params['reg_lambda'] = round(max(reg_lambda,0),4) #double
    params['learning_rate'] = round(max(learning_rate,0),4) #double
    params['num_iterations'] = int(round(num_iterations,0)) #int
    params['early_stopping_rounds'] = int(round(early_stopping_rounds,0)) #int
    return params

def bayes_parameter_opt_lgb(_feat, _target, init_round=15, opt_round=25, _seed=42):
    warnings.filterwarnings("ignore")
    def lgb_eval(max_depth, num_leaves, min_child_samples, bagging_fraction, bagging_freq, feature_fraction, reg_alpha, reg_lambda, learning_rate, num_iterations, early_stopping_rounds):
      para = SetParameter(max_depth, num_leaves, min_child_samples, bagging_fraction, bagging_freq, feature_fraction, reg_alpha, reg_lambda, learning_rate, num_iterations, early_stopping_rounds)
      lgbData  = lgb.Dataset(pd_train[_feat], pd_train[_target])
      lgbmodel = lgb.train(para, lgbData,  valid_sets=[lgbData] ,verbose_eval=False )
      # pd_test
      _prob = lgbmodel.predict(pd_test[_feat])
      roc_auc = roc_auc_score(pd_test['target'], _prob)
      return roc_auc

    lgbBO = BayesianOptimization(lgb_eval, {'max_depth': (2, 16),
                                            'num_leaves': (2, 64),
                                            'min_child_samples': (2, 64),
                                            'bagging_fraction': (0.1, 0.9),
                                            'bagging_freq': (0, 16),
                                            'feature_fraction': (0.1, 0.9),
                                            'reg_alpha': (0.05, 0.9),
                                            'reg_lambda': (0.05, 0.9),
                                            'learning_rate': (0.01, 0.1),
                                            'num_iterations' : (100, 200),
                                            'early_stopping_rounds' : (5, 20),
                                              }, random_state=_seed)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    # return best parameters
    warnings.filterwarnings('default')
    return lgbBO
opt_params = bayes_parameter_opt_lgb(_feat = df_feature[df_feature.selected==1].feature.to_list()
                                                        , _target = 'target'
                                                        , init_round=60, opt_round=20, _seed=123)

print(f"{'=='*50}\n",SetParameter(**opt_params.max['params']),f"\n{'=='*50}")
hyper_parameter = pd.DataFrame(opt_params.res)
hyper_parameter = pd.concat([hyper_parameter['target'], hyper_parameter['params'].apply(pd.Series)], axis=1)


|   iter    |  target   | max_depth | num_le... | min_ch... | baggin... | baggin... | featur... | reg_alpha | reg_la... | learni... | num_it... | early_... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
| 2         | 0.6721402 | 11.750568 | 19.740638 | 16.064790 | 0.5410518 | 11.511503 | 0.4384851 | 0.8836495 | 0.6321052 | 0.0532838 | 139.21175 | 10.147670 |
| 3         | 0.6545147 | 12.206695 | 29.191479 | 5.7000295 | 0.4184354 | 11.807926 | 0.2459933 | 0.1991339 | 0.5018186 | 0.0578644 | 163.44009 | 17.741476 |
| 4         | 0.6632749 | 12.142374 | 39.883457 | 46.791489 | 0.3583671 | 5.7886184 | 0.2826105 | 0.2996569 | 0.5863297 | 0.0182894 | 143.37011 | 11.462941 |
| 5         | 0.6627310 | 8.9115913 | 28.401477 | 21.360195 | 0.4410810 | 14.294226 | 0.8553280 | 0.4765611 | 0.5803600 | 0.0204056 | 131.72854 | 11.222393 |
| 6         | 0.6714758 | 14.128328 | 17.528232 | 31

## Final model

In [18]:
from typing_extensions import final
#Training initial model
feat_final = df_feature[df_feature.selected==1].feature.to_list()
final_para = SetParameter(**hyper_parameter[hyper_parameter.target == hyper_parameter.target.max()].drop(columns="target").to_dict(orient='records')[0])
lgb_train = lgb.Dataset(pd_train[feat_final], label=pd_train['target'])
lgb_eval = lgb.Dataset(pd_test[feat_final], label=pd_test['target'], reference=lgb_train)

print(f"Training model with {len(feat_final)} feature : {feat_final} \nwith parameter :{final_para}")
final_model = lgb.train(final_para,  lgb_train,  valid_sets=[lgb_eval])

print("\n\nPredict and Model Performance")
pd_train['pred_prob'], pd_train['pred_bin'] = pdPredict(_df=pd_train, _model=final_model)
pd_test['pred_prob'], pd_test['pred_bin'] = pdPredict(_df=pd_test, _model=final_model)
pd_oot['pred_prob'], pd_oot['pred_bin'] = pdPredict(_df=pd_oot, _model=final_model)

df_importance = pd.DataFrame({'feature':final_model.feature_name() , 'importance':final_model.feature_importance()}).sort_values('importance', ascending=False)
print(f"\n\nTop 10 feature importance from initial model")
display(df_importance)


Training model with 32 feature : ['num_children', 'age', 'income', 'maxosdc_last_30d', 'dcspend_last_30d', 'easypymt_last_30d', 'avg_savaccbal_30d', 'avg_currentaccbal_30d', 'mob', 'outflow30d', 'payroll', 'have_cc', 'have_acc_planet', 'ntu_flag', 'occ_sa', 'occ_se', 'occ_gov', 'occ_prof', 'occ_ent', 'occ_frl', 'occ_hwf', 'occ_stu', 'occ_une', 'seg_lmass', 'seg_umass', 'gender_female', 'sta_single', 'avgsav2inflow', 'sav_stability', 'netflow2inflow', 'bank_sav', 'bank_prod'] 
with parameter :{'boosting_type': 'gbdt', 'objective': 'binary', 'metric': ['auc'], 'verbose': -1, 'max_depth': 4, 'num_leaves': 6, 'min_child_samples': 45, 'bagging_fraction': 0.9, 'bagging_freq': 6, 'feature_fraction': 0.9, 'reg_alpha': 0.05, 'reg_lambda': 0.9, 'learning_rate': 0.1, 'num_iterations': 114, 'early_stopping_rounds': 13}
[1]	valid_0's auc: 0.640074
Training until validation scores don't improve for 13 rounds
[2]	valid_0's auc: 0.647975
[3]	valid_0's auc: 0.655562
[4]	valid_0's auc: 0.657932
[5]	vali

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[11]	valid_0's auc: 0.661592
[12]	valid_0's auc: 0.663154
[13]	valid_0's auc: 0.665647
[14]	valid_0's auc: 0.669225
[15]	valid_0's auc: 0.67021
[16]	valid_0's auc: 0.670602
[17]	valid_0's auc: 0.670735
[18]	valid_0's auc: 0.671517
[19]	valid_0's auc: 0.671043
[20]	valid_0's auc: 0.671667
[21]	valid_0's auc: 0.67119
[22]	valid_0's auc: 0.672103
[23]	valid_0's auc: 0.672057
[24]	valid_0's auc: 0.67178
[25]	valid_0's auc: 0.673232
[26]	valid_0's auc: 0.673522
[27]	valid_0's auc: 0.673268
[28]	valid_0's auc: 0.673427
[29]	valid_0's auc: 0.673107
[30]	valid_0's auc: 0.673011
[31]	valid_0's auc: 0.67311
[32]	valid_0's auc: 0.673576
[33]	valid_0's auc: 0.674321
[34]	valid_0's auc: 0.675231
[35]	valid_0's auc: 0.675779
[36]	valid_0's auc: 0.675803
[37]	valid_0's auc: 0.676467
[38]	valid_0's auc: 0.67742
[39]	valid_0's auc: 0.677557
[40]	valid_0's auc: 0.677989
[41]	valid_0's auc: 0.678389
[42]	valid_0's auc: 0.677739
[43]	valid_0's auc: 0.677451
[44]	valid_0's auc: 0.677299
[45]	valid_0's auc:

,feature,importance
8,mob,64
1,age,46
6,avg_savaccbal_30d,39
3,maxosdc_last_30d,30
28,sav_stability,30
5,easypymt_last_30d,25
27,avgsav2inflow,25
29,netflow2inflow,22
2,income,19
9,outflow30d,16


#Model - Evaluate

In [29]:
lst_result = []
for tbNam, tbpd in zip(['Training', 'Test-in-time', 'Out-of-time'],[pd_train, pd_test, pd_oot]):
  pct_target = tbpd['target'].mean()
  roc_auc = roc_auc_score(tbpd['target'], tbpd['pred_prob'])
  gini = 2*roc_auc -1
  tmp_dict = {'data_set':tbNam, '%target':pct_target, 'roc_auc':roc_auc, 'gini':gini }
  lst_result.append(tmp_dict)
print(f"Model performance")
display(pd.DataFrame(lst_result))
print(f"\n\nFeature process")
df_feature.merge(df_importance.rename(columns={'importance':'final_importance'}), on='feature', how='left').fillna(0)

Model performance


,data_set,%target,roc_auc,gini
0,Training,0.050188,0.729699,0.459399
1,Test-in-time,0.012135,0.682281,0.364562
2,Out-of-time,0.011427,0.739883,0.479766




Feature process


,feature,count,mean,stddev,min,25%,50%,75%,max,importance,selected,final_importance
0,obs_id,37539,107588.72974772903,62377.74124081243,1,53589,107246,161656,215993,0.0,0,0.0
1,target,37539,0.05018780468312955,0.2183351983314606,0,0,0,0,1,0.0,0,0.0
2,num_children,37539,-63134.293428168035,792069.4730762646,-9999999.0,0.0,0.0,0.0,5.0,1.0,1,3.0
3,age,37539,-63096.861131090336,792072.4568699166,-9999999.0,29.0,36.0,45.0,61.0,49.0,1,46.0
4,income,37539,-92202.79700657756,1041360.0828098088,-9999999.0,4364.98,12324.44,21345.0,198000.0,10.0,1,19.0
5,maxosdc_last_30d,37539,-97975.68425743919,1042180.3773554043,-9999999.0,0.0,0.0,4056.29,2084410.06,18.0,1,30.0
6,dcspend_last_30d,37539,-108994.38270065801,1039453.6403301145,-9999999.0,0.0,0.0,0.0,1541316.58,11.0,1,8.0
7,easypymt_last_30d,37539,-108074.22467300674,1039518.4874098451,-9999999.0,77.0,374.0,925.0,999982.0,18.0,1,25.0
8,savacc_bal,37539,-89281.38697621174,1045652.6419708276,-9999999.0,5.63,768.6,7476.5,4414296.66,13.0,0,0.0
9,currentacc_bal,37539,-109219.73259809798,1039373.3965960342,-9999999.0,0.0,0.0,0.0,1.0,0.0,0,0.0


In [30]:
print(f"%target by bin vin out-of-time (avg target : {pd_oot.target.mean():6f})\n\n")
pd_oot.groupby('pred_bin').agg(min_prob=('pred_prob','min')
                                            , max_prob=('pred_prob','max')
                                            , nobs=('obs_id','count')
                                            , pct_target=('target', 'mean')
                               ).sort_values('pred_bin', ascending=False)

%target by bin vin out-of-time (avg target : 0.011427)




<ipython-input-30-1680127740>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd_oot.groupby('pred_bin').agg(min_prob=('pred_prob','min')


,min_prob,max_prob,nobs,pct_target
pred_bin,,,,
10,0.071044,0.294325,2354,0.039507
9,0.051938,0.071012,2354,0.022515
8,0.042592,0.051931,2354,0.014019
7,0.036116,0.042591,2353,0.009775
6,0.030897,0.036115,2355,0.006369
5,0.026567,0.030896,2354,0.006372
4,0.022886,0.026565,2354,0.006797
3,0.019525,0.022886,2354,0.004673
2,0.015545,0.019522,2354,0.003823


#Presentation

In [ ]:
pd_oot.replace([-9999999],[np.nan])[pd_oot.pred_bin==10].describe()

,obs_id,target,num_children,age,income,maxosdc_last_30d,dcspend_last_30d,easypymt_last_30d,savacc_bal,currentacc_bal,avg_savaccbal_30d,avg_currentaccbal_30d,mob,inflow30d,outflow30d,inflow1_15,outflow1_15,net_flow_30d,net_flow_15d,payroll,have_cc,have_acc_planet,ntu_flag,occ_sa,occ_se,occ_gov,occ_prof,occ_ent,occ_frl,occ_hwf,occ_stu,occ_une,seg_lmass,seg_mass,seg_umass,gender_female,sta_single,avgsav2incm,avgsav2inflow,debitspend2inflow,savings_ratio,sav_stability,netflow2inflow,bank_sav,bank_cur,bank_prod,pred_prob
count,2353.000000,2353.000000,2353.00000,2353.000000,2353.000000,2353.000000,2353.000000,2.353000e+03,2.353000e+03,2353.000000,2353.000000,2353.000000,2353.000000,2.353000e+03,2.353000e+03,2.353000e+03,2.353000e+03,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.0,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2353.000000,2352.000000,1817.000000,1817.000000,2270.000000,2270.000000,1817.000000,2353.0,2353.000000,2353.000000,2353.000000
mean,108779.970251,0.044624,0.01870,31.852529,15339.699630,4604.728729,198.639086,1.383835e+03,8.603126e+03,0.000425,4326.503612,0.005482,45.087123,3.494158e+04,3.632689e+04,3.494158e+04,3.632689e+04,-1385.309078,-1385.309078,0.253294,0.051424,0.011050,0.0,0.453889,0.200170,0.015300,0.003825,0.000850,0.082448,0.010200,0.208670,0.024649,0.602210,0.369741,0.028049,0.442839,0.815130,8.994239,0.609179,0.005226,1.000000,482.795371,-1.201451,1.0,0.000425,1.316192,0.084660
std,63287.035967,0.206521,0.15862,8.523922,17226.898801,22053.690037,3678.695949,2.173539e+04,4.229562e+04,0.020615,20798.239522,0.265937,30.598572,8.391995e+04,8.392605e+04,8.391995e+04,8.392605e+04,13616.379654,13616.379654,0.434990,0.220907,0.104558,0.0,0.497975,0.400212,0.122768,0.061741,0.029148,0.275104,0.100499,0.406444,0.155087,0.489546,0.482837,0.165149,0.496827,0.388275,291.418547,12.378876,0.050218,0.000023,4862.180384,25.774964,0.0,0.020615,0.511248,0.015284
min,68.000000,0.000000,0.00000,23.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.550000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-424281.640000,-424281.640000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000044,0.000244,0.000000,0.998903,0.016136,-1043.453125,1.0,0.000000,1.000000,0.068581
25%,52435.000000,0.000000,0.00000,26.000000,4364.980000,0.000000,0.000000,1.960000e+02,4.973000e+01,0.000000,399.220000,0.000000,20.000000,1.500000e+03,2.499120e+03,1.500000e+03,2.499120e+03,-913.310000,-913.310000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.038777,0.013078,0.000000,1.000000,0.297534,-0.086073,1.0,0.000000,1.000000,0.073518
50%,108265.000000,0.000000,0.00000,28.000000,11557.550000,0.000000,0.000000,3.810000e+02,1.011720e+03,0.000000,1078.380000,0.000000,44.000000,1.400000e+04,1.611792e+04,1.400000e+04,1.611792e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.109891,0.040105,0.000000,1.000000,0.990472,-0.001544,1.0,0.000000,1.000000,0.080323
75%,164759.000000,0.000000,0.00000,36.000000,19629.340000,2375.280000,0.000000,8.480000e+02,6.356120e+03,0.000000,2673.650000,0.000000,65.000000,3.300000e+04,3.482689e+04,3.300000e+04,3.482689e+04,7.410000,7.410000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.417634,0.120262,0.000000,1.000000,8.892560,0.005059,1.0,0.000000,2.000000,0.090852
max,215972.000000,1.000000,3.00000,60.000000,179503.820000,437739.830000,167347.420000,1.043790e+06,1.369385e+06,1.000000,619954.370000,12.

In [ ]:
pd_oot.replace([-9999999],[np.nan])[pd_oot.pred_bin==1].describe()

,obs_id,target,num_children,age,income,maxosdc_last_30d,dcspend_last_30d,easypymt_last_30d,savacc_bal,currentacc_bal,avg_savaccbal_30d,avg_currentaccbal_30d,mob,inflow30d,outflow30d,inflow1_15,outflow1_15,net_flow_30d,net_flow_15d,payroll,have_cc,have_acc_planet,ntu_flag,occ_sa,occ_se,occ_gov,occ_prof,occ_ent,occ_frl,occ_hwf,occ_stu,occ_une,seg_lmass,seg_mass,seg_umass,gender_female,sta_single,avgsav2incm,avgsav2inflow,debitspend2inflow,savings_ratio,sav_stability,netflow2inflow,bank_sav,bank_cur,bank_prod,pred_prob
count,2429.000000,2429.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.000000,2035.0,2035.000000,2035.0,2035.000000,2035.000000,2035.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2429.000000,2024.000000,0.0,0.0,1570.000000,1568.000000,0.0,2429.00000,2429.000000,2429.000000,2429.000000
mean,106775.275422,0.001235,0.080098,44.038821,11083.002496,300.215941,4.451651,36.207371,143.809754,0.001474,91.484015,0.614226,36.660934,0.0,0.024157,0.0,0.024157,-0.024157,-0.024157,0.094689,0.014409,0.003294,0.162207,0.380403,0.254426,0.033347,0.001647,0.000823,0.102100,0.020173,0.003705,0.040758,0.469329,0.348703,0.019761,0.400988,0.503911,0.188075,NaN,NaN,0.998720,67.344171,NaN,0.66900,0.000823,0.782215,0.018009
std,63025.195214,0.035129,0.330329,7.059445,12835.419873,910.961447,106.110910,74.020262,481.588739,0.038376,140.932636,27.487445,59.763883,0.0,1.089757,0.0,1.089757,1.089757,1.089757,0.292845,0.119195,0.057307,0.368716,0.485586,0.435628,0.179578,0.040555,0.028689,0.302842,0.140620,0.060770,0.197769,0.499161,0.476658,0.139207,0.490200,0.500088,2.494481,NaN,NaN,0.035681,841.752248,NaN,0.47067,0.028689,0.614149,0.000903
min,148.000000,0.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,-49.160000,-49.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.033125,NaN,0.00000,0.000000,0.000000,0.015071
25%,50968.000000,0.000000,0.000000,39.000000,2379.585000,0.000000,0.000000,0.000000,0.070000,0.000000,0.335000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000032,NaN,NaN,1.000000,1.000000,NaN,0.00000,0.000000,0.000000,0.017695
50%,105490.000000,0.000000,0.000000,43.000000,6464.350000,0.000000,0.000000,0.000000,4.460000,0.000000,17.320000,0.000000,13.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.002824,NaN,NaN,1.000000,1.000000,NaN,1.00000,0.000000,1.000000,0.018065
75%,162332.000000,0.000000,0.000000,49.000000,15140.150000,20.540000,0.000000,41.500000,74.885000,0.000000,125.285000,0.000000,51.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.024335,NaN,NaN,1.000000,2.657458,NaN,1.00000,0.000000,1.000000,0.018672
max,215888.000000,1.000000,3.000000,61.000000,135000.000000,8900.420000,3666.910000,373.000000,8210.010000,1.000000,643.390000,1239.950000,887.000000,0.0,49.160000,0.0,49.160000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,83.620000,NaN,NaN,1.000000,24994.000000,NaN,1.00000,1.000000,3.000000,0.019596
